In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
#Preprocessing
https://www.kaggle.com/code/kritanjalijain/twitter-sentiment-analysis-lstm/notebook

In [3]:
test_train_set=pd.read_csv('data/data_identification.csv')
emotion=pd.read_csv('data/emotion.csv')


In [4]:
test_str1="@Hope I love U"
test_str2="Dad good night"
test_str3="@Donald gooooooooooood joob"

In [5]:
def text_cleaning(text):
    tmp=[]
    for t in text.split(' '):
        if t.startswith('@') or t.startswith('<LH>'):
            continue
        else:
            tmp.append(t)
    clean_text= " ".join(tmp)
    return clean_text

In [6]:
text_cleaning(test_str1)

'I love U'

In [7]:
# https://officeguide.cc/python-read-write-json-encode-decode-tutorial-examples/
#load json會把輸入字串轉為字典
i=0
tweet_id=[]
text=[]
with open('data/tweets_DM.json', "r") as f:
    for line in f:
        #print(line)
        data=json.loads(line)
        tweet_id.append(data['_source']['tweet']['tweet_id'])
        clean_text=text_cleaning(data['_source']['tweet']['text'])
        text.append(clean_text)
        #print(clean_text)
        '''i=i+1
        if i==10:
            break'''

In [8]:
dict = {'tweet_id': tweet_id, 'text': text} 
tweet_text = pd.DataFrame(dict)

In [9]:
tweet_text['text'][:20]

0     People who post "add me on #Snapchat" must be ...
1     As we see, Trump is dangerous to #freepress ar...
2     Confident of your obedience, I write to you, k...
3                        Now ISSA is stalking Tasha 😂😂😂
4     "Trust is not the same as faith. A friend is s...
5     Thx for the BEST TIME tonight. What stories! H...
6               Still waiting on those supplies Liscus.
7                              Love knows no gender. 😢😭
8     More highlights are being shown than actual sp...
9     When do you have enough ? When are you satisfi...
10    The #SSM debate; (a manufactured fantasy used ...
11    I love suffering 🙃🙃 I love when valium does no...
12    Can someone tell my why my feeds scroll back t...
13    You know you research butterflies when predict...
14    My brother didn't tell me he was going to horr...
15    On a scale of Kylie Jenner-Heidi Klum, I consi...
16    Progress at House Meyer, Pre Galv sub frame go...
17          Vomi post birthday celebrations! #la

In [10]:
test_set=test_train_set[test_train_set['identification']=='test']
train_set=test_train_set[test_train_set['identification']=='train']

In [11]:

#train+emotion
train_emo=pd.merge(train_set,emotion,on='tweet_id')
train=pd.merge(tweet_text,train_emo,on='tweet_id')
test=pd.merge(test_set,tweet_text,on='tweet_id')


In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer()
tok.fit_on_texts(pd.concat([train,test],ignore_index=True)['text'])
vocab_size = len(tok.word_index) + 1
print(vocab_size)

2022-11-21 00:41:12.918908: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 00:41:13.232348: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-21 00:41:13.268666: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-21 00:41:13.268706: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

716744


In [13]:

train_encoded_phrase = tok.texts_to_sequences(train['text']) 
test_encoded_phrase = tok.texts_to_sequences(test['text'])

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(train_encoded_phrase, maxlen=59,padding='post',truncating='post')
X_test = pad_sequences(test_encoded_phrase, maxlen=59,padding='post',truncating='post')
#print(X_train[:5])

In [15]:
X_train.shape

(1455563, 59)

In [16]:
from sklearn.preprocessing import OneHotEncoder
'''
Labels turn into one-hot encoding
'''

encoder = OneHotEncoder(handle_unknown='ignore')



In [17]:
reshaped_emotion=np.reshape(list(train['emotion']),(-1,1))
encoder.fit(reshaped_emotion)

OneHotEncoder(handle_unknown='ignore')

In [18]:
y_train=encoder.transform(reshaped_emotion)

In [19]:
y_train.toarray()

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [48]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Bidirectional
from keras.layers import Flatten, Dropout
from keras.layers import Embedding,SpatialDropout1D
# create the model
embedding_vecor_length = 50

# Initialising the model
model_1 = Sequential()


model_1.add(Embedding(vocab_size, embedding_vecor_length, input_length=59))
model_1.add(Dropout(0.2))

model_1.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model_1.add(MaxPooling1D(pool_size=2))
model_1.add((LSTM(32)))
model_1.add(Dropout(0.4))

# Adding output layer
model_1.add(Dense(8, activation='softmax'))

# Printing the model summary
print(model_1.summary())

# Compiling the model
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 59, 50)            35837200  
                                                                 
 dropout_10 (Dropout)        (None, 59, 50)            0         
                                                                 
 conv1d_5 (Conv1D)           (None, 59, 32)            4832      
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 29, 32)           0         
 1D)                                                             
                                                                 
 lstm_5 (LSTM)               (None, 32)                8320      
                                                                 
 dropout_11 (Dropout)        (None, 32)                0         
                                                      

In [49]:
num_epochs = 1
# Fitting the data to the model
history_1 = model_1.fit(X_train, y_train.toarray(),epochs=num_epochs , batch_size=128 ,verbose=1)

11372/11372 [==============================] - 1463s 128ms/step - loss: 1.3685 - accuracy: 0.5103


In [50]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(train['emotion'])

LabelEncoder()

In [51]:
def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

In [52]:
pred_result = model_1.predict(X_test, batch_size=128)
pred_result[:5]

3219/3219 [==============================] - 17s 5ms/step


array([[0.01585974, 0.0819651 , 0.04996465, 0.02478131, 0.4345113 ,
        0.08357871, 0.02996697, 0.27937216],
       [0.05203462, 0.08985004, 0.22372973, 0.0318957 , 0.19283545,
        0.29875436, 0.03912314, 0.07177692],
       [0.02201701, 0.15466182, 0.06586471, 0.02379698, 0.43625316,
        0.0944984 , 0.02650981, 0.17639808],
       [0.03795854, 0.08552589, 0.15318269, 0.02463756, 0.36469474,
        0.18568859, 0.03206505, 0.11624696],
       [0.06315969, 0.04113721, 0.4264006 , 0.01959665, 0.10975022,
        0.25091007, 0.03398763, 0.05505786]], dtype=float32)

In [53]:
pred_result = label_decode(label_encoder, pred_result)
pred_result[:10]

array(['joy', 'sadness', 'joy', 'joy', 'disgust', 'joy', 'trust', 'joy',
       'anticipation', 'sadness'], dtype=object)

In [54]:
test['emotion']=pred_result

In [55]:
output=pd.DataFrame([])
output['id']=pd.DataFrame(test['tweet_id'])
output['emotion']=pred_result
output

,id,emotion
0,0x28cc61,joy
1,0x2db41f,sadness
2,0x2466f6,joy
3,0x23f9e9,joy
4,0x1fb4e1,disgust
...,...,...
411967,0x2c4dc2,joy
411968,0x31be7c,anticipation
411969,0x1ca58e,anticipation
411970,0x35c8ba,joy


In [56]:
path = 'output_data.csv'
#df = pd.DataFrame(columns=headers, data=my2dlist)
output.to_csv(path, index=False) # index = True